# Tutorial - QFrame

## What is a QFrame?
QFrame is a class which generates an SQL statement. It stores fields info in `QFrame.data` parameter which is a dictionary.

`QFrame.data` has `select` key in which it stores `fields` which we want to have in our SQL statement. Each key have to have specified `type` which can be 'dim' if the varibale is a dimension variable or 'num' if the variable is a numeric variable. Let's take a look at all options that we can have under `select` and `fields` keys.

```json
{
  "select": {
    "table": "table",
    "schema": "schema",
    "fields": {
      "column": {
        "type": "dim",
        "as": "",
        "group_by": "",
        "order_by": "",
        "expression": "",
        "select": "",
        "custom_type": ""
      }
    },
    "where": "",
    "distinct": "",
    "having": "",
    "limit": ""
  }
}
```

- `table` - Name of the table.
- `schema` - Name of the schema.
- `fields`, in each field:
    - `type` - Type of the column. Options:

        - 'dim' - VARCHAR(500)  
        - 'num' - FLOAT
     
     Every column has to have specified type. If you want to sepcify another type check `custom_type`.
    - `as` - Column alias (name).

    - `group_by` - Aggregation type. Possibilities:

        - 'group' - This field will go to GROUP BY statement.
        - {'sum', 'count', 'min', 'max', 'avg'} - This field will by aggregated in specified way.
  
     If you don't want to aggregate fields leave `group_by` empty in each field.
    - `order_by` - Put the field in order by statement. Options:
    
        - 'ASC'
        - 'DESC'
        
    - `expression` - Expression, eg. CASE statement, column operation, CONCAT statement, ... .
    - `select` - Set 0 if you don't want to put this field in SELECT statement.
    - `custom_type` - Specify custom SQL data type, eg. DATE.
- `where` - Add where statement, eg. 'sales>100'
- `distinct` - Set 1 to add distinct to select
- `having` - Add having statement, eg. 'sum(sales)>100'
- `limit` - Add limit, eg. 100

## How to create a QFrame?
You can create a QFrame manually - passing the data directly to QFrame or automatically - using `initiate` function.

In [1]:
from grizly import (
    get_path, 
    QFrame
)

### Manually - using dictionary

This method is the most direct method of creating a QFrame - to use it you need to know the structure of `QFrame.data`. From following dictionary

In [2]:
data = {
  "select": {
    "table": "table",
    "schema": "schema",
    "fields": {
      "col": {
        "type": "dim"
      }
    }
  }
}

QFrame will generate a simple sql

In [3]:
qf = QFrame().from_dict(data)
print(qf)

SELECT col
FROM schema.table


Here we also used simple method `.get_sql()` which prints sql saved in QFrame.

### Manually - using JSON file

We use a `.json` file to conviniently manipulate information about columns, renames and other things that might be very verbose to manipulate in python code. We can edit the json file into a json editor like http://jsoneditoronline.org/ more conviniently than in Python code.

After editing the `store.json` we can read it back inside a QFrame using `read_json()`.

This means we can use our json as our main `store` of verbose information and python as our main way to manipulate said information.

In [4]:
json_path = get_path("grizly_dev", "notebooks", "store.json")
qf.save_json(json_path=json_path, subquery="my_query_1")

qf = QFrame().from_json(json_path=json_path, subquery="my_query_1")
print(qf)

Data saved in /home/analyst/grizly_dev/notebooks/store.json
SELECT col
FROM schema.table


### Automatically - using from_table method

The other way to generate a QFrame is to use `QFrame.from_table()` method.

In [5]:
qf = QFrame(engine="mssql+pyodbc://redshift_acoe", db='redshift', interface="pyodbc")
qf.from_table(schema="administration", table="table_tutorial")

print(qf)

SELECT col1,
       col2,
       col3,
       col4
FROM administration.table_tutorial


In [6]:
qf_c = QFrame(engine="mssql+pyodbc://redshift_acoe", db='redshift', interface="pyodbc")
qf_c.from_table(schema="administration", table="table_tutorial", columns=["col1", "col3"])

print(qf_c)

SELECT col1,
       col3
FROM administration.table_tutorial


You can also specify which columns you want to pull from database by using `columns` parameter.

## Working with the QFrame
There is a lot of methods which you can use to edit the QFrame. You can check them in QFrame docs. In this tutorial we will only show some of them.

### Checking QFrame size

You can check how many rows returns the query generated by your QFrame by using Python build-in `len()` function.

In [7]:
len(qf)

2

### Doing some basic SQL stuff
Let's now add a `where` statement, rename some fields, add calculated field and remove some fields`.

In [8]:
qf.query("col2 > 1") #<- where
qf.rename({"col1": "items", "col2": "price"})
qf.assign(calculated_field="col4*2", 
          type='num', 
          custom_type='double precision')
qf.remove(["col3", "col4"])
print(qf)

SELECT col1 AS "items",
       col2 AS "price",
       col4*2 AS "calculated_field"
FROM administration.table_tutorial
WHERE col2 > 1


:Be aware that `rename()` method doesn't change the name of the field but only the alias (final name) of the column.:

Now you can check how the data changed calling `data` attribute.

In [9]:
qf.data

{'select': {'table': 'table_tutorial',
  'schema': 'administration',
  'fields': {'col1': {'type': '',
    'as': 'items',
    'group_by': '',
    'order_by': '',
    'expression': '',
    'select': '',
    'custom_type': 'character varying(500)'},
   'col2': {'type': '',
    'as': 'price',
    'group_by': '',
    'order_by': '',
    'expression': '',
    'select': '',
    'custom_type': 'double precision'},
   'calculated_field': {'type': 'num',
    'as': 'calculated_field',
    'group_by': '',
    'order_by': '',
    'expression': 'col4*2',
    'custom_type': 'double precision'}},
  'engine': '',
  'where': 'col2 > 1',
  'distinct': '',
  'having': '',
  'offset': '',
  'limit': '',
  'sql_blocks': {'select_names': ['col1 as "items"',
    'col2 as "price"',
    'col4*2 as "calculated_field"'],
   'select_aliases': ['items', 'price', 'calculated_field'],
   'group_dimensions': [],
   'group_values': [],
   'order_by': [],
   'types': ['CHARACTER VARYING(500)',
    'DOUBLE PRECISION',
 

You can see that now we also have `sql_blocks` key. You can ignore it. This key is used to build SQL statement and is generated any time `get_sql()` method is called.

### Forking

Forking qframes can be important if your data workflow needs to take the same sql table and apply different transformations to it.

Sometimes we want to fork, do some transforms, then union the QFrames back together which results into an append operation on the data side.

Let's create two copies of one QFrame.

In [10]:
qf1 = qf.copy()
qf2 = qf.copy()

## Unioning data

There are two ways of unioning two QFrames - we can union by the position of the field or by the final name of the columns (that means the alias). 

In [11]:
from grizly import union

qf1.rename({"col2": "price_1", "calculated_field": "price_2"})
qf2.rename({"col2": "price_2", "calculated_field": "price_1"})

#### Union by the positon

In [12]:
uqf_pos = union(qframes=[qf1, qf2], union_type="UNION ALL", union_by='position')
print(uqf_pos)

Data unioned successfully.
SELECT col1 AS "items",
       col2 AS "price_1",
       col4*2 AS "price_2"
FROM administration.table_tutorial
WHERE col2 > 1
UNION ALL
SELECT col1 AS "items",
       col2 AS "price_2",
       col4*2 AS "price_1"
FROM administration.table_tutorial
WHERE col2 > 1


#### Union by the column names

In [13]:
uqf_name = union(qframes=[qf1, qf2], union_type="UNION ALL", union_by='name')
print(uqf_name)

Data unioned successfully.
SELECT col1 AS "items",
       col2 AS "price_1",
       col4*2 AS "price_2"
FROM administration.table_tutorial
WHERE col2 > 1
UNION ALL
SELECT col1 AS "items",
       col4*2 AS "price_1",
       col2 AS "price_2"
FROM administration.table_tutorial
WHERE col2 > 1


You can see that in this case union changes the order of the columns. 

## Joining data

In [14]:
from grizly import join

We will be using `Chinook.sqlite` to visualize data.

In [15]:
engine_string = "sqlite:///" + get_path("grizly_dev", "tests", "Chinook.sqlite")

### Simple join

First table is `Track` table.

In [16]:
tracks = {  'select': {
                'fields': {
                    'TrackId': { 'type': 'dim'},
                    'Name': {'type': 'dim'},
                    'AlbumId': {'type': 'dim'},
                    'Composer': {'type': 'dim'},
                    'UnitPrice': {'type': 'num'}
                },
                'table': 'Track'
            }
}
tracks_qf = QFrame(engine=engine_string, db="sqlite").from_dict(tracks)
print(tracks_qf)

SELECT TrackId,
       Name,
       AlbumId,
       Composer,
       UnitPrice
FROM Track


In [17]:
tracks_qf.to_df().sample(5)

,TrackId,Name,AlbumId,Composer,UnitPrice
2280,2281,My Melancholy Blues,186,Mercury,0.99
3313,3314,One For The Road,258,E. Schrody/L. Dimant/L. Muggerud,0.99
2199,2200,Porch,181,Eddie Vedder,0.99
1331,1332,Hooks In You,105,Adrian Smith/Bruce Dickinson,0.99
2130,2131,Miss Sarajevo,176,"Brian Eno, Bono, Adam Clayton, The Edge & Larr...",0.99


The second table is `PlaylistTrack` table. 

In [18]:
playlist_track_qf = QFrame(engine=engine_string, db="sqlite").from_table(table="PlaylistTrack")

print(playlist_track_qf)

SELECT PlaylistId,
       TrackId
FROM PlaylistTrack


In [19]:
playlist_track_qf.to_df().sample(5)

,PlaylistId,TrackId
1045,1,1113
492,1,3452
8652,15,3407
459,1,655
1997,1,686


Now let's join them on `TrackId`.

In [20]:
joined_qf = join([tracks_qf,playlist_track_qf], 
                 join_type="left join", 
                 on="sq1.TrackId=sq2.TrackId")

print(joined_qf)

Data joined successfully.
SELECT sq1.TrackId AS "TrackId",
       sq1.Name AS "Name",
       sq1.AlbumId AS "AlbumId",
       sq1.Composer AS "Composer",
       sq1.UnitPrice AS "UnitPrice",
       sq2.PlaylistId AS "PlaylistId"
FROM
  (SELECT TrackId,
          Name,
          AlbumId,
          Composer,
          UnitPrice
   FROM Track) sq1
LEFT JOIN
  (SELECT PlaylistId,
          TrackId
   FROM PlaylistTrack) sq2 ON sq1.TrackId=sq2.TrackId


In [21]:
joined_qf.to_df().sample(5)

,TrackId,Name,AlbumId,Composer,UnitPrice,PlaylistId
3830,1538,Naquele Dia,124,João Suplicy,0.99,1
3239,1306,The Number Of The Beast,103,None,0.99,8
2098,859,Se...,69,None,0.99,8
5048,2054,Selvagem,167,Bi Ribeiro/Herbert Vianna/João Barone,0.99,1
7245,2946,When I Look At The World,233,"Adam Clayton, Bono, Larry Mullen, The Edge",0.99,8


As you can see in this example `UnitPrice` is taken from the first table. By default join function is taking all fields from the first QFrame, then all the fields from the second QFrame which are not in the first and so on. If you still want to keep all fields from each QFrame we have to set `unique_col=False`. We will see in the next example how does it work.

### Multiple join

Now let's use one more table to check how does multiple join look like.

In [22]:
playlists_qf = QFrame(engine=engine_string, db="sqlite").from_table(table="Playlist")

print(playlists_qf)

SELECT PlaylistId,
       Name
FROM Playlist


In [23]:
playlists_qf.to_df().sample(5)

,PlaylistId,Name
16,17,Heavy Metal Classic
10,11,Brazilian Music
11,12,Classical
5,6,Audiobooks
17,18,On-The-Go 1


Now if we want to join `Tracks`, `PlaylistTrack` and `Playlist` tables we can use `TrackId` and `PlaylistId`. We can see that in `Tracks` and `Playlist` tables we have the same column `Name`. Let's check the option `unique_col=False` and analyse duplicated columns.

In [24]:
joined_qf = join(qframes=[tracks_qf, playlist_track_qf, playlists_qf], 
                 join_type=['left join', 'left join'], 
                 on=['sq1.TrackId=sq2.TrackId', 'sq2.PlaylistId=sq3.PlaylistId'], 
                 unique_col=False)

Data joined successfully.
Please remove or rename duplicated columns. Use your_qframe.show_duplicated_columns() to check duplicates.


In [25]:
joined_qf.show_duplicated_columns()

 DUPLICATED COLUMNS: 
 
TrackId:	 ['sq1.TrackId', 'sq2.TrackId']

Name:	 ['sq1.Name', 'sq3.Name']

PlaylistId:	 ['sq2.PlaylistId', 'sq3.PlaylistId']

Use your_qframe.remove() to remove or your_qframe.rename() to rename columns.


We can see that three columns occure in two different tables. We will remove `sq2.TrackId` and  `sq2.PlaylistId` fields and rename `Name` column.

In [26]:
joined_qf.remove(['sq2.TrackId', 
                  'sq2.PlaylistId']).rename({'sq1.Name': 'TrackName', 
                                             'sq3.Name': 'PlaylistType'})
print(joined_qf)

SELECT sq1.TrackId AS "TrackId",
       sq1.Name AS "TrackName",
       sq1.AlbumId AS "AlbumId",
       sq1.Composer AS "Composer",
       sq1.UnitPrice AS "UnitPrice",
       sq3.PlaylistId AS "PlaylistId",
       sq3.Name AS "PlaylistType"
FROM
  (SELECT TrackId,
          Name,
          AlbumId,
          Composer,
          UnitPrice
   FROM Track) sq1
LEFT JOIN
  (SELECT PlaylistId,
          TrackId
   FROM PlaylistTrack) sq2 ON sq1.TrackId=sq2.TrackId
LEFT JOIN
  (SELECT PlaylistId,
          Name
   FROM Playlist) sq3 ON sq2.PlaylistId=sq3.PlaylistId


In [27]:
joined_qf.to_df().sample(5)

,TrackId,TrackName,AlbumId,Composer,UnitPrice,PlaylistId,PlaylistType
6641,2691,You Got Me Rocking,218,Jagger/Richards,0.99,8,Music
5125,2087,Me Liga,169,None,0.99,1,Music
5661,2305,Binky The Doormat,189,Bill Berry-Peter Buck-Mike Mills-Michael Stipe,0.99,1,Music
2795,1125,I Stand Alone,88,Sully Erna,0.99,8,Music
2412,968,Zombie Eaters,77,Faith No More,0.99,8,Music


## Pivot

Again we will use `Chinook.sqlite` and `Track` table to visualize data.

In [28]:
engine_string = "sqlite:///" + get_path("grizly_dev", "tests", "Chinook.sqlite")

In [29]:
qf = QFrame(engine=engine_string, db="sqlite").from_table(table="Track")

len(qf)

3503

Our table has `3503` rows - we will limit the data to `15` rows to get better view. We will use `QFrame.window()` method to be sure that the result is deterministic. 

In [30]:
qf.window(offset=100, limit=30, order_by=["TrackId"])
qf.to_df()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,101,Be Yourself,11,1,4,"Cornell, Commerford, Morello, Wilk",279484,9106160,0.99
1,102,Doesn't Remind Me,11,1,4,"Cornell, Commerford, Morello, Wilk",255869,8357387,0.99
2,103,Drown Me Slowly,11,1,4,"Cornell, Commerford, Morello, Wilk",233691,7609178,0.99
3,104,Heaven's Dead,11,1,4,"Cornell, Commerford, Morello, Wilk",276688,9006158,0.99
4,105,The Worm,11,1,4,"Cornell, Commerford, Morello, Wilk",237714,7710800,0.99
5,106,Man Or Animal,11,1,4,"Cornell, Commerford, Morello, Wilk",233195,7542942,0.99
6,107,Yesterday To Tomorrow,11,1,4,"Cornell, Commerford, Morello, Wilk",273763,8944205,0.99
7,108,Dandelion,11,1,4,"Cornell, Commerford, Morello, Wilk",278125,9003592,0.99
8,109,#1 Zero,11,1,4,"Cornell, Commerford, Morello, Wilk",299102,9731988,0.99
9,110,The Curse,11,1,4,"Cornell, Commerford, Morello, Wilk",309786,10029406,0.99


In [31]:
qf.pivot(rows=["Composer"], columns=["AlbumId", "GenreId"], values="UnitPrice", aggtype="sum")
qf.to_df()

,Composer,11_4,12_5,13_2
0,"Berry Gordy, Jr./Janie Bradford",0.0,0.99,0.00
1,Bert Russell/Phil Medley,0.0,0.99,0.00
2,Billy Cobham,0.0,0.00,6.93
3,Bo Diddley,0.0,0.99,0.00
4,Brian Holland/Freddie Gorman/Georgia Dobbins/R...,0.0,0.99,0.00
5,Chuck Berry,0.0,1.98,0.00
6,"Cornell, Commerford, Morello, Wilk",9.9,0.00,0.00
7,Eddie Cochran/Jerry Capehart,0.0,0.99,0.00
8,"Enotris Johnson/Little Richard/Robert ""Bumps"" ...",0.0,0.99,0.00
9,George Duke,0.0,0.00,0.99


As you can see all values in `AlbumId` and `GenreId` became separate columns, `Composer` column has been grouped and `UnitPrice` has been sumed up.